In [1]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import gradio as gr

c:\Users\sivat\Anaconda3\envs\genai-env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def model_chat_inference(model_name):
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

    while True: 
        input_text = input('You: ')
        print('You: ',input_text)
        if input_text.lower() in ['quit','exit','bye','good bye']:
            print('Chatbot: Good bye!',)
            break 
        inputs = tokenizer.encode(input_text,return_tensors='pt')
        outputs = model.generate(inputs,max_new_tokens = 80)
        response = tokenizer.decode(outputs[0], skip_special_tokens=True).strip()
        print(f'Chatbot: ',response)
    return tokenizer,model

### Blenderbot

In [3]:
blenderbot = "facebook/blenderbot-400M-distill"   
blenderbot_tokenizer,blenderbot_model = model_chat_inference(blenderbot)

You:  bye
Chatbot: Good bye!


### T5 base

In [4]:
t5basebot = "google/flan-t5-base"
t5basebot_tokenizer, t5basebot_model = model_chat_inference(t5basebot)

You:  bye
Chatbot: Good bye!


### Bart base

In [5]:
bartbot = "facebook/bart-base"
bartbot_tokenizer,bartbot_model = model_chat_inference(bartbot)

You:  bye
Chatbot: Good bye!


### T5 small

In [6]:
t5smallbot = "google/flan-t5-small"
t5smallbot_tokenizer,t5smallbot_model = model_chat_inference(t5smallbot)

You:  bye
Chatbot: Good bye!


## Single Chat Interface

In [7]:
# blenderbot = "facebook/blenderbot-400M-distill"
# blenderbot_tokenizer = AutoTokenizer.from_pretrained(blenderbot)
# blenderbot_model = AutoModelForSeq2SeqLM.from_pretrained(blenderbot)

In [8]:
# Chat function with state (history) and blenderbot is used for chat
def chat_blenderbot(message, history):
    inputs = blenderbot_tokenizer.encode(message, return_tensors='pt')
    outputs = blenderbot_model.generate(inputs)
    response = blenderbot_tokenizer.decode(outputs[0], skip_special_tokens=True).strip()
    history.append({"role": "user", "content": message})
    history.append({"role": "assistant", "content": response})
    print(history)
    return "", history

# Build the Gradio Blocks UI
with gr.Blocks(title="BlenderBot Chatbot",theme=gr.themes.Glass()) as demo:
    gr.Markdown("## 🤖 BlenderBot 400M Chatbot")

    chatbot = gr.Chatbot(type='messages')
    message_input = gr.Textbox(placeholder="Type a message...", label="Your Message")
    clear_btn = gr.Button("Clear")

    state = gr.State([])

    message_input.submit(chat_blenderbot, inputs=[message_input, state], outputs=[message_input, chatbot])
    clear_btn.click(lambda: [], outputs=[chatbot, state])

# Launch the app
demo.launch()


* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


## Parallel Chatbots Response Comparision Interface

In [10]:
# Define chat functions for each bot
def chat_bot(message, history,tokenizer = None,model = None):
    inputs = tokenizer.encode(message, return_tensors='pt')
    outputs = model.generate(inputs)
    response = tokenizer.decode(outputs[0], skip_special_tokens=True).strip()
    history.append({"role": "user", "content": message})
    history.append({"role": "assistant", "content": response})
    return history

from concurrent.futures import ThreadPoolExecutor

def respond_all(message, chatbot_blenderbot, chatbot_t5, chatbot_bart, chatbot_t5small):
    with ThreadPoolExecutor() as executor:
        futures = [
            executor.submit(chat_bot, message, chatbot_blenderbot, blenderbot_tokenizer, blenderbot_model),
            executor.submit(chat_bot, message, chatbot_t5, t5basebot_tokenizer, t5basebot_model),
            executor.submit(chat_bot, message, chatbot_bart, bartbot_tokenizer, bartbot_model),
            executor.submit(chat_bot, message, chatbot_t5small, t5smallbot_tokenizer, t5smallbot_model),
        ]

        results = [f.result() for f in futures]

    return "", *results

# # use this code if parallel threading is not possible
# def respond_all(message, chatbot_blenderbot, chatbot_t5, chatbot_bart, chatbot_t5small):

#     blenderbot_response_history = chat_bot(message, chatbot_blenderbot,
#                                            tokenizer = blenderbot_tokenizer,model = blenderbot_model)
    
#     t5_base_response_history = chat_bot(message, chatbot_t5, tokenizer =t5basebot_tokenizer,model = t5basebot_model)

#     bartbot_response_history = chat_bot(message,chatbot_bart,
#                                         tokenizer = bartbot_tokenizer,model = bartbot_model)
#     t5_small_response_history = chat_bot(message,chatbot_t5small,
#                                          tokenizer = t5smallbot_tokenizer,model = t5smallbot_model)
#     return "", blenderbot_response_history, t5_base_response_history, bartbot_response_history, t5_small_response_history

with gr.Blocks(title="Compare Chatbots",theme = gr.themes.Glass()) as demo:
    gr.Markdown("## Comparing BlenderBot and T5")

    with gr.Row():
        chatbot_blenderbot = gr.Chatbot(label="BlenderBot", height=400,type ='messages')
        state_blenderbot = gr.State([])
        chatbot_t5 = gr.Chatbot(label="T5", height=400,type = 'messages')
        state_t5 = gr.State([])
        chatbot_bart = gr.Chatbot(label = "BartBase",height = 400,type = 'messages')
        state_bart = gr.State([])
        chatbot_t5small = gr.Chatbot(label = "T5-Small",height =400,type = 'messages')
        state_t5small = gr.State([])

    with gr.Row():
        message_input = gr.Textbox(placeholder="Type a message...", label="Your Message")
    with gr.Row():
        send_button = gr.Button("Send to All")
        clear_button = gr.Button("Clear All")

    send_button.click(respond_all,
                     inputs=[message_input, state_blenderbot, state_t5, state_bart, state_t5small],
                     outputs=[message_input, chatbot_blenderbot, chatbot_t5, chatbot_bart, chatbot_t5small])
    
    clear_button.click(
        lambda: ([], [], [], [], [], [], [], []),
        outputs=[chatbot_blenderbot, chatbot_t5, chatbot_bart, chatbot_t5small,
                state_blenderbot, state_t5, state_bart, state_t5small
        ]
    )

demo.launch()

* Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.


c:\Users\sivat\Anaconda3\envs\genai-env\lib\site-packages\transformers\generation\utils.py:1249: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


In [ ]:
# # CODE for not using threading and simple running

# def respond_both(message, chatbot_blenderbot, chatbot_t5, chatbot_bart, chatbot_t5small):
#     # blenderbot_history = chatbot_blenderbot[:]  # Create a copy to avoid modifying the state directly
#     # t5_history = chatbot_t5[:]

#     blenderbot_response_history = chat_bot(message, chatbot_blenderbot,
#                                            tokenizer = blenderbot_tokenizer,model = blenderbot_model)
    
#     t5_base_response_history = chat_bot(message, chatbot_t5, tokenizer =t5basebot_tokenizer,model = t5basebot_model)

#     bartbot_response_history = chat_bot(message,chatbot_bart,
#                                         tokenizer = bartbot_tokenizer,model = bartbot_model)
#     t5_small_response_history = chat_bot(message,chatbot_t5small,
#                                          tokenizer = t5smallbot_tokenizer,model = t5smallbot_model)
#     return "", blenderbot_response_history, t5_base_response_history, bartbot_response_history, t5_small_response_history